# Amazon Dataset Data Cleaning

## Table of Contents

1. [Project Overview](#project-overview)
2. [Dataset Description](#dataset-description)
3. [Initial Data Exploration](#initial-data-exploration)
4. [Data Cleaning Steps](#data-cleaning-steps)
    - [Handling Missing Values](#handling-missing-values)
    - [Removing Duplicates](#removing-duplicates)
    - [Standardizing Formats](#standardizing-formats)
    - [Feature Engineering](#feature-engineering)
    - [Standardizing and Normalizing Categorical Values](#standardizing-and-normalizing-categorical-values)
5. [Summary](#summary)


## 1. Project Overview
This notebook demonstrates the process of cleaning and preparing an Amazon product dataset for analysis. The goal is to address common data quality issues and produce a dataset ready for further analytics or modeling.


## 2. Dataset Description
- **Rows:** 1000
- **Columns:** 55
- **Features:** Product details, prices, categories, reviews, and more.
- **Source:** https://github.com/luminati-io/eCommerce-dataset-samples/tree/main


## 3. Initial Data Exploration
Let's load the data and examine its structure, types, and missing values.


In [181]:
import numpy as np
import pandas as pd
import re

from matplotlib import pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings("ignore")

# Load your data
df = pd.read_csv("amazon-products.csv")  # Update with your actual file path

# Preview the data
df.head()

,timestamp,title,seller_name,brand,description,initial_price,final_price,currency,availability,reviews_count,...,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin
0,2023-08-08 00:00:00.000,Saucony Men's Kinvara 13 Running Shoe,Orv███tor███,Saucony,"When it comes to lightweight speed, nothing cr...",NaN,"""57.79""",USD,In Stock,702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-09 00:00:00.000,Kishigo Premium Black Series Heavy Duty Unisex...,Ama███.co███,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,NaN,"""28.5""",USD,In Stock,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-04 00:00:00.000,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,Twi███uxe███,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,"""49.99""","""33.99""",USD,In Stock,3178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:00:00.000,Accutire MS-4021B Digital Tire Pressure Gauge ...,Cit███ran███Dir██████,Accutire,About this item Heavy duty construction and ru...,1.795000000000000e+01,1.795000000000000e+01,USD,In Stock,8034,...,Automotive,Tire Repair Tools,50.00,NaN,"[{""subcategory_name"":""Automotive"",""subcategory...",False,NaN,NaN,NaN,NaN
4,2024-01-16 00:00:00.000,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,PRA███ EN███PRI███,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,"""1299""","""799""",INR,In stock,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Types and Missing Values
Check the data types and count missing values in each column.


In [184]:
df.info()
df.isnull().sum().sort_values(ascending=False).head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             1000 non-null   object 
 1   title                 1000 non-null   object 
 2   seller_name           826 non-null    object 
 3   brand                 999 non-null    object 
 4   description           998 non-null    object 
 5   initial_price         814 non-null    object 
 6   final_price           997 non-null    object 
 7   currency              1000 non-null   object 
 8   availability          994 non-null    object 
 9   reviews_count         1000 non-null   int64  
 10  categories            1000 non-null   object 
 11  asin                  1000 non-null   object 
 12  buybox_seller         926 non-null    object 
 13  number_of_sellers     984 non-null    float64
 14  root_bs_rank          999 non-null    float64
 15  answered_questions    

country_of_origin    1000
prices_breakdown     1000
product_details      1000
ingredients           954
images                948
badge                 924
upc                   901
root_bs_category      894
amazon_choice         894
subcategory_rank      894
dtype: int64

## 4. Data Cleaning Steps
This section covers the main cleaning operations performed on the dataset.


#### a. Handling Missing Values

- Checked for missing values in all columns.
- Filled missing `initial_price` with `final_price` where possible.
- Dropped columns with all null values or redacted data (e.g., `product_details`, `prices_breakdown`, `country_of_origin`, `seller_nam`).
.
.


In [188]:
# Check for missing values
print(df.isna().sum())

#missing_percentage = (df.isnull().sum() / len(df)) * 100
#print(missing_percentage)

#to fill the null values in initial price column with final price
df['initial_price'] = df['initial_price'].fillna(df['final_price'])
#df['initial_price'].isnull().sum()

df['final_price'].isnull().sum()

# Drop columns with all nulls or redacted and url's as the information is redundant in this project
df = df.drop(columns=['seller_name', 'product_details', 'prices_breakdown', 'country_of_origin', 'images', 'image_url', 'url', 'origin_url'])

timestamp                  0
title                      0
seller_name              174
brand                      1
description                2
initial_price            186
final_price                3
currency                   0
availability               6
reviews_count              0
categories                 0
asin                       0
buybox_seller             74
number_of_sellers         16
root_bs_rank               1
answered_questions        31
domain                     0
images_count               0
url                        0
video_count                0
image_url                  5
item_weight              247
rating                     0
product_dimensions        67
seller_id                  0
date_first_available      79
discount                 179
model_number             108
manufacturer              30
department               338
plus_content               0
upc                      901
video                      0
top_review                 8
variations    

### b. Removing Duplicates

We check for and remove duplicate rows to ensure data integrity.


In [191]:
# Check for duplicates
df.duplicated().sum()

# Remove duplicates if any
#df = df.drop_duplicates()


np.int64(0)

#### c. Standardizing Formats

- Cleaned and standardized string columns by removing unwanted characters and whitespace.
- Standardized price columns to float.
- Standardized domain URLs and parsed date columns.
s.



In [194]:
# Clean string columns
for col in ['variations', 'delivery', 'features', 'subcategory_rank']:
    df[col] = df[col].str.replace('[', '', regex=False)
    df[col] = df[col].str.replace(']', '', regex=False)
    df[col] = df[col].str.replace('{', '', regex=False)
    df[col] = df[col].str.replace('}', '', regex=False)
    df[col] = df[col].str.replace('"', '', regex=False)
    df[col] = df[col].str.strip()

# Standardize domain
df['domain'] = df['domain'].str.strip()
df['domain'] = df['domain'].apply(lambda x: f"https://{x}/" if x.startswith('www.') else x)

# Convert price columns from scientific/string notation to float
def clean_price(price):
    if pd.isna(price):
        return np.nan
    if isinstance(price, str):
        # Remove quotes and dollar signs if present
        return float(price.replace('"', '').replace('$', '').replace(',', '').replace('₹', '').replace('£', ''))
    return float(price)

 
# Apply the function to price columns
df['initial_price'] = df['initial_price'].apply(lambda x: clean_price(x) if pd.notna(x) else x)
df['final_price'] = df['final_price'].apply(lambda x: clean_price(x) if pd.notna(x) else x)

# Standardize string columns
df['title'] = df['title'].str.strip()
df['brand'] = df['brand'].str.strip()
df['availability'] = df['availability'].str.strip()
df['availability'] = df['availability'].replace({'In Stock': 'In Stock', 'In Stock.':'In Stock', 'In stock':'In Stock'})
df['categories'] = df['categories'].str.strip()

# Parse timestamp, date_first_available column
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date_first_available'] = pd.to_datetime(df['date_first_available'], errors='coerce')

pd.options.display.float_format = '{:.2f}'.format


In [196]:
df['availability'].value_counts()

availability
In Stock                                       709
Only 1 left in stock - order soon               49
Only 2 left in stock - order soon               29
Only 5 left in stock - order soon               15
In stock Usually ships within 2 to 3 days.      13
                                              ... 
Only 14 left in stock - order soon.              1
Only 9 left in stock (more on the way).          1
Only 13 left in stock (more on the way)          1
In stock. Usually ships within 3 to 4 days.      1
Only 4 left in stock (more on the way)           1
Name: count, Length: 72, dtype: int64

#### d. Feature Engineering

- Created new features such as `discount_percentage`, `discount_cleaned`, and `discount_amount`.
- Extracted year and month from the timestamp.
- Split and cleaned category columns into hierarchical levels.
- Cleaned and merged item weight information from multiple col
- To ensure consistency between `initial_price`, `final_price`, and `discount_cleaned`, we:
- Impute missing `final_price` using `initial_price` and `discount_cleaned` where possible.
- Fix discrepancies between the calculated and existing `final_price`.
- Update `discount_amount` after corrections.umns.


In [155]:
# Discount percentage
df['discount_percentage'] = ((df['initial_price'] - df['final_price']) / df['initial_price']) * 100
df['discount_percentage'] = df['discount_percentage'].round(2)


# Clean discount column
def clean_discount(discount):
    if pd.isna(discount):
        return None
    match = re.search(r'(\d+)%', str(discount)) or re.search(r'$$(\d+)%$$', str(discount)) or re.search(r'(\d+(\.\d+)?)', str(discount))  
    # Extract numeric value
    return float(match.group(1)) if match else None

# Calculate discount amount
def calculate_discount_amount(row):
    if pd.notna(row['initial_price']) and pd.notna(row['final_price']):
        return row['initial_price'] - row['final_price']
    return None

df['discount_amount'] = df.apply(calculate_discount_amount, axis=1)
df['discount_cleaned'] = df['discount'].apply(clean_discount)


# Extract year and month from timestamp
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month

# Split categories into subcategories
df['categories'] = df['categories'].str.replace('[', '', regex=False)
df['categories'] = df['categories'].str.replace(']', '', regex=False)
df['categories'] = df['categories'].str.replace('"', '', regex=False)
df['main_category'] = df['categories'].str.split(',').str[0]
df['sub_category_1'] = df['categories'].str.split(',').str[1]
df['sub_category_2'] = df['categories'].str.split(',').str[2].fillna('')
df['sub_category_3'] = df['categories'].str.split(',').str[3].fillna('')
df['sub_category_1'] = df['sub_category_1'].str.strip()
df['sub_category_2'] = df['sub_category_2'].str.strip()
df['sub_category_3'] = df['sub_category_3'].str.strip()


In [157]:
# Clean and merge item weight
def extract_weight_from_dimensions(dim_str):
    if not isinstance(dim_str, str):
        return None
    weight_patterns = [
        r'(\d+(\.\d+)?)\s*Pounds?',
        r'(\d+(\.\d+)?)\s*Ounces?',
        r'(\d+(\.\d+)?)\s*Grams',
        r'(\d+(\.\d+)?)\s*Kilograms',
        r'(\d+(\.\d+)?)\s*g\b'
    ]
    for pattern in weight_patterns:
        match = re.search(pattern, dim_str, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            unit = match.group(0).lower()
            if 'pound' in unit:
                return value
            elif 'ounce' in unit:
                return value / 16
            elif 'gram' in unit or 'g' in unit:
                return value / 453.592
            elif 'kilogram' in unit:
                return value * 2.20462
    return None

def clean_item_weight(weight_str):
    if pd.isna(weight_str):
        return None
    match = re.search(r'(\d+(\.\d+)?)\s*(\w+)', str(weight_str))
    if match:
        value = float(match.group(1))
        unit = match.group(3).lower()
        if 'pound' in unit or unit == 'lbs':
            return value
        elif 'ounce' in unit or unit == 'oz':
            return value / 16
        elif 'gram' in unit or unit == 'g':
            return value / 453.592
        elif 'kilogram' in unit or unit == 'kg':
            return value * 2.20462
    return None

df['weight_from_dimensions'] = df['product_dimensions'].apply(extract_weight_from_dimensions)
df['cleaned_item_weight'] = df['item_weight'].apply(clean_item_weight)

def merge_weights(row):
    if pd.notna(row['cleaned_item_weight']):
        return row['cleaned_item_weight']
    elif pd.notna(row['weight_from_dimensions']):
        return row['weight_from_dimensions']
    return None

df['merged_item_weight'] = df.apply(merge_weights, axis=1)
df['final_item_weight'] = df['merged_item_weight'].apply(lambda x: f"{x:.2f} pounds" if pd.notna(x) else None)


In [159]:
def fix_nan_final_price(row):
    if pd.isna(row['final_price']) and pd.notna(row['discount_cleaned']):
        return row['initial_price'] * (1 - row['discount_cleaned'] / 100)
    return row['final_price']

# To ensure consistency between `initial_price`, `final_price`, and `discount_cleaned`
def fix_final_price_discrepancies(row):
    if pd.notna(row['discount_cleaned']):
        calculated_final_price = row['initial_price'] * (1 - row['discount_cleaned'] / 100)
        # Check if there's a significant difference between calculated and existing final price
        if pd.isna(row['final_price']) or abs(row['final_price'] - calculated_final_price) > 0.01:
            return calculated_final_price
    return row['final_price']

# Apply both fixes
df['final_price'] = df.apply(fix_nan_final_price, axis=1)
df['final_price'] = df.apply(fix_final_price_discrepancies, axis=1)

# Update discount_amount after fixing final_price
df['discount_amount'] = df['initial_price'] - df['final_price']


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   timestamp               1000 non-null   datetime64[ns]
 1   title                   1000 non-null   object        
 2   brand                   999 non-null    object        
 3   description             998 non-null    object        
 4   initial_price           1000 non-null   float64       
 5   final_price             1000 non-null   float64       
 6   currency                1000 non-null   object        
 7   availability            994 non-null    object        
 8   reviews_count           1000 non-null   int64         
 9   categories              1000 non-null   object        
 10  asin                    1000 non-null   object        
 11  buybox_seller           926 non-null    object        
 12  number_of_sellers       984 non-null    float64  

### e. Standardizing and Normalizing Categorical Values

To ensure consistency and usability, we standardize categorical columns such as `availability`, `currency`, and `categories`.

#### Standardize Availability
We normalize all variants of "in stock" and similar phrases to "In Stock".


In [175]:
def standardize_availability(val):
    if pd.isna(val):
        return None
    val = val.strip().lower()
    if "in stock" in val:
        return "In Stock"
    elif "out of stock" in val:
        return "Out of Stock"
    elif "unavailable" in val:
        return "Unavailable"
    else:
        return val.title()  # fallback: capitalize first letter

df['availability'] = df['availability'].apply(standardize_availability)


In [179]:
df['availability'].value_counts()

availability
In Stock                              982
Usually Ships Within 1 To 2 Months      2
Usually Ships Within 6 To 10 Days.      2
Available To Ship In 1-2 Days           2
Usually Ships Within 7 To 8 Days        1
Usually Ships Within 7 Days.            1
Usually Ships Within 8 To 9 Days        1
Usually Ships Within 7 Days             1
Usually Ships Within 1 To 2 Weeks       1
Usually Ships Within 2 To 3 Days        1
Name: count, dtype: int64

## 5. Summary

- The Amazon product dataset has been thoroughly cleaned and standardized.
- Key data cleaning steps included:
  - Handling missing values and removing duplicates.
  - Standardizing categorical columns such as `availability`, `currency`, and hierarchical `category` columns.
  - Parsing and normalizing price and discount information.
  - Extracting and unifying product weight data.
  - Splitting and cleaning category columns into `main_category`, `sub_category_1`, `sub_category_2`, and `sub_category_3` for better analysis.
- The resulting dataset is now consistent, well-structured, and ready for further analysis.